In [1]:
pip install mysql-connector-python pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import mysql.connector  
import pandas as pd  

# List of CSV file paths
csv_file_paths = [
    'movies_adventure_data.csv',
    'movies_animation_data.csv',
    'movies_crime_data.csv',
    'movies_fantasy_data.csv',
    'movies_biography_data.csv'
]

# data from the CSV files
df_list = []

# CSV file and load the data into a DataFrame
for file_path in csv_file_paths:
    try:
        # Read the CSV file into a DataFrame (table-like structure)
        df = pd.read_csv(file_path)
        
        # Handle missing data by replacing NaN with 0
        df['Duration'] = df['Duration'].fillna(0)
        df['Rating'] = df['Rating'].fillna(0)
        df['Vote Count'] = df['Vote Count'].fillna(0)

        # Add the DataFrame (table) to the list
        df_list.append(df)
        print(f"Successfully loaded data from {file_path}")

    except FileNotFoundError:
        # If the file doesn't exist, print an error message
        print(f"File {file_path} not found.")
    except Exception as e:
        # If there's any other error, print it
        print(f"Error reading {file_path}: {e}")

# Combine all DataFrames into one large DataFrame
final_df = pd.concat(df_list, ignore_index=True)

# Connect to MySQL database
connection = mysql.connector.connect(
    host='localhost',  
    user='root',       
    password='root', 
    database='IMBD_Movies_project1'  
)

# interact with the database
cursor = connection.cursor()

# data into the 'movie' table
insert_query = """
    INSERT INTO movie (Name, Genre, Duration, Rating, `Vote Count`)
    VALUES (%s, %s, %s, %s, %s)
"""

# Prepare the data for insertion 
data = final_df[['Name', 'Genre', 'Duration', 'Rating', 'Vote Count']].values.tolist()

try:
    
    cursor.executemany(insert_query, data)
    
    connection.commit()

    print("CSV data has been successfully loaded into MySQL.")

except mysql.connector.Error as err:
    
    print(f"Error: {err}")
    connection.rollback()

finally:
    
    cursor.close()
    connection.close()

File movies_adventure_data.csv not found.
File movies_animation_data.csv not found.
File movies_crime_data.csv not found.
File movies_fantasy_data.csv not found.
File movies_biography_data.csv not found.


ValueError: No objects to concatenate